智能体的六种设计模式之一：Chain
Chain，也就是串联工作模式，并用这个agent来示例如何写议论文，从列提纲到写内容到润色修改，三个任务串联在一起，分别调用大模型，以实现更高的写作质量。


In [ ]:
from openai import OpenAI
from datetime import datetime
import json
from typing import List, Dict, Callable
import os
import re
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

In [ ]:

llm_name = llm_name = "qwen-plus"

def call_llm(user_prompt, system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = client.chat.completions.create(
        model=llm_name,
        messages=messages
    )
    
    return response.choices[0].message.content


In [ ]:
def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = call_llm(f"{prompt}\n{result}")
        print(result)
    
    return result



In [ ]:
writing_processing_steps = [
    """
    请根据以下主题撰写一篇议论文的大纲，
    包含引言、三个到五个主要论点，以及结论。
    每个论点附带简要的解释或论据，
    主题如下：
    """,
    
    """
    请根据以下大纲撰写一篇完整的议论文，
    要求语言流畅、逻辑清晰、论点充分，
    每个段落围绕大纲内容展开，
    字数为1000字左右。
    大纲如下：
    """,

    """
    请对以下议论文进行全文润色，包括语法检查、用词优化和句式调整，
    使文章语言自然流畅、逻辑清晰、表达生动而简洁，避免生硬和刻板的 AI 风格。
    保持原文观点不变。
    文章如下：
    """
]


In [ ]:
result=chain("人工智能时代更需要学习编程技术",writing_processing_steps)

智能体的六种设计模式之二：Parallelization
Parallelization，也就是并行工作模式，并用这个agent来示例如何将一个复杂任务分拆成多个关联不大的子任务，同时调用大模型接口，再汇总结果。
mapreduce模式，是将一个复杂任务分拆成多个关联不大的子任务，然后调用大模型接口，再汇总结果。



In [ ]:
from openai import OpenAI
from datetime import datetime
import json
from typing import List, Dict, Callable
import os
import re
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)



In [ ]:
llm_name = llm_name = "qwen-plus"

def call_llm(user_prompt, system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = client.chat.completions.create(
        model=llm_name,
        messages=messages
    )
    
    return response.choices[0].message.content


多线程
否则要等很久

In [ ]:
def parallel(inputs: List[str], n_workers: int = 3) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(call_llm, prompt) for prompt in inputs]
        return [f.result() for f in futures]


In [ ]:
baijia=["儒家","法家","道家","墨家"]

In [ ]:
def gen_prompt(aspect, topic):
    prompt = f"""
    针对如下问题进行思考分析，需要一步步分析展示思考过程，并得出结论。
    问题如下：{topic}。
    你分析的框架和角度如下：{aspect}
    """
    return prompt


In [ ]:
prompts = [gen_prompt(x, topic="to be or not to be") for x in baijia]


In [ ]:
impact_results = parallel(prompts, n_workers=len(prompts))


In [ ]:
for result in impact_results:
    print('-------------')
    print(result)

智能体的六种设计模式之三：Routing
Routing，也就是路由工作模式，Routing（路由） 根据输入内容的特征或条件，动态地将任务分配给最合适的子任务或大模型的一种工作流模式。它强调智能决策和灵活调度，适用于多样化输入和多策略处理的场景。

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from openai import OpenAI
from datetime import datetime
import json
from typing import List, Dict, Callable
import os
import re
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
llm_name = llm_name = "qwen-plus"

def call_llm(user_prompt, system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = client.chat.completions.create(
        model=llm_name,
        messages=messages
    )
    
    return response.choices[0].message.content


In [ ]:
def extract_xml(text: str, tag: str) -> str:
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""



In [ ]:
writing_prompt = {
    "议论文": """
    撰写一篇议论文，围绕以下主题展开明确的观点，并提供有力的论据支持。
    要求：
    1. **观点鲜明**：开篇提出明确的论点。
    2. **逻辑严谨**：通过事实、数据、案例进行论证。
    3. **结构清晰**：包括引言、论证、结论三部分。
    4. **语言有说服力**：条理分明，语言简洁有力。
    """,
    
    "散文": """
    请写一篇散文，围绕以下主题抒发真挚细腻的情感，描绘生动的画面。
    要求：
    1. **情感真挚**：用细腻的文字表达内心感受。
    2. **意境优美**：通过比喻、拟人等修辞，营造氛围。
    3. **语言流畅**：语言自然，富有节奏感。
    4. **不拘结构**：可以自由组织段落和内容。
    """,
    
    "诗歌": """
    请创作一首诗歌，围绕以下主题，用简洁生动的语言表达情感或描绘画面。
    要求：
    1. **意象丰富**：用生动的意象表现主题。
    2. **语言凝练**：短句精练，富有节奏感。
    3. **情感浓烈**：真挚自然地传达内心情感。
    """,
    
    "小说": """
    请创作一篇小说，围绕以下主题展开故事情节，塑造生动的角色，并营造引人入胜的氛围。
    要求：
    1. **情节完整**：包括开端、发展、高潮、结局。
    2. **人物鲜明**：塑造富有个性和深度的角色。
    3. **氛围生动**：通过细节描写和环境铺垫增强代入感。
    4. **语言流畅**：叙述清晰，富有表现力。
    """
}


In [ ]:
def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""

    selector_prompt = f"""
    你是一个作家，请根据用户输入的主题，判断其需要的写作风格，并将内容路由到对应模块。
    输入的主题是 {input}
    路由的分类包括：
    {list(routes.keys())}
    你需要用简洁的语言归纳给出分类的理由，然后输出分类名称。都需要输出到 XML 格式标签中。

    <reasoning>
    分类的理由
    </reasoning>

    <selection>
    分类名称
    </selection>
    """.strip()

    route_response = call_llm(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()
    print("分类的理由")
    print(reasoning)
    print(f"\n分类名称: {route_key}")

    # Process input with selected specialized prompt
    selected_prompt = routes[route_key]
    return call_llm(f"{selected_prompt}\n要求字数五百字，主题是: {input}")

例子

In [ ]:
requests = ["为什么要学习编程", "天边的云"]



In [ ]:
for request in requests:
    print("-" * 40)
    print(request)
    print("-" * 40)
    response = route(request, writing_prompt)
    print(response)


智能体的六种设计模式之四：Evaluator-optimizer
Evaluator-optimizer，也就是评估优化工作模式，它是由一个大模型负责生成内容，另一个大模型负责进行评估和反馈。这样多轮迭代以优化大模型输出效果。


In [ ]:
from openai import OpenAI
import os
import re
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
llm_name = "qwen-plus"

def call_llm(user_prompt, system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = client.chat.completions.create(
        model=llm_name,
        messages=messages
    )
    
    return response.choices[0].message.content


In [ ]:
def extract_xml(text: str, tag: str) -> str:
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""


In [ ]:
generator_prompt = """
任务：
根据提供的主题写一篇论证文章。请遵循以下结构和步骤，确保文章逻辑严密、有说服力。

1. 引言部分
开头简要介绍背景信息，说明该议题的重要性。清晰明确地提出文章的中心论点，即你要支持的观点。

2. 主体部分
主体部分应分为多个段落，每个段落围绕一个理由展开，支撑你的中心论点。

3. 结论部分
总结你的中心论点和主要支持理由，提醒读者文章的关键观点。

4. 注意事项
保持语言简洁明了，避免过于复杂或模糊的表达。
确保论证严谨，避免情绪化的语言或空洞的说法。
使用可靠的来源来支持论点，避免无根据的主张。
字数要求1000字左右。

如果有任何的反馈，你需要基于反馈来修改。

你需要使用下面的XML标签来输出两部分内容。

<thoughts>
这里写你对主题的理解，对反馈内容的理解，以及如何来修改内容。
</thoughts>

<response>
论文内容写在这里
</response>
"""


In [ ]:
evaluator_prompt = """
请根据以下标准，评估输入的论证文章的质量和逻辑性：

1. 明确性与中心论点：
论证的中心论点是否清晰明确？是否能够轻松识别并理解论点？
论证是否围绕一个可辨识的主题展开，避免陈述无争议的事实？

2. 论证的事实性与前提：
文章的前提是否真实且可靠？前提是否有明确的证据支持？
是否存在任何事实错误或不准确的描述？

3. 推理与逻辑性：
论证的推理是否合理？是否能从前提出发，合逻辑地推导出结论？
是否存在逻辑谬误，例如黑白谬误、以偏概全等？

4. 证据支持与相关性：
论证的证据是否充分且相关？每个支持理由是否有可靠的证据（例如专家意见、数据、案例）支持？
是否有证据断层或刻意被遗漏？证据是否直接支持论点？

5. 一致性与逻辑结构：
论证是否存在自相矛盾的地方？前提和结论之间是否一致？
论证的结构是否合理，段落之间是否有逻辑过渡，支持理由是否清晰组织？

6. 批判性思维标准：
清晰性：表述是否简洁明了？
精准性：语言是否具体准确，避免模糊表述？
相关性：所有内容是否与论点紧密相关？
完整性：论证是否考虑了所有相关证据？
公平性：辩论者是否公平处理反对意见，是否无偏见？

7. 反对策略与对反对意见的回应：
论证是否回应了反对意见？如果有，回应是否充分且合逻辑？
论证是否忽略了相关证据，或者存在关键的反对意见没有被讨论？

8. 结论的支持：
论证的结论是否得到前提和证据的充分支持？结论是否合理、可靠？

根据上述标准，总体评价该论证文章的质量，给出两部分反馈内容。
根据文章的论证质量输出 **评分**，评分是 0 到 100 的区间，
指出论证中的 **弱点**，并给出改进的建议。

你需要使用下面的 XML 标签格式来输出评分和反馈：

<evaluation>
这里输出论文评分
</evaluation>

<feedback>
这里输出反馈建议
</feedback>
"""


In [ ]:
task = """
AI的迅速发展会带来大量失业吗
"""


In [ ]:
def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""
    full_prompt = f"{prompt}\n{context}\n文章主题如下:{task}" if context else f"{prompt}\n文章主题如下:{task}"
    response = call_llm(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")

    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")

    return thoughts, result


In [ ]:
def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\n文章主题如下:{task}\n要评估的内容如下: {content}"
    response = call_llm(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")

    print("=== EVALUATION START ===")
    print(f"Score: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===")

    return evaluation, feedback


In [ ]:
def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []

    # 生成初稿
    thoughts, result = generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})

    # 进入迭代
    while True:
        # 评估文章内容
        evaluation, feedback = evaluate(evaluator_prompt, result, task)

        # 评估通过则退出
        if float(evaluation) >= 90:
            return result, chain_of_thought

        # 将之前内容和反馈进行拼接，作为背景知识
        context = "\n".join([
            "之前的写作内容如下：",
            *[f"- {m}" for m in memory],
            f"\n反馈如下：\n{feedback}"
        ])

        # 根据反馈重新生成
        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})


In [ ]:
result = loop(task, evaluator_prompt, generator_prompt)

辩论场景

In [ ]:
pos_prompt = """
你是一个辩论场上的选手，你需要根据提供的正方观点进行论证。
保持语言简洁明了，避免过于复杂或模糊的表达。
确保论证严谨，避免情绪化的语言或空洞的说法。
使用可靠的来源来支持论点，避免无根据的主张。

如果辩论对方有任何的观点意见，你需要回应对方的观点意见并修改自己的论证。你的目标不是赢，而是探寻真理。

你需要使用下面的XML标签来输出。

<response>
正方观点:
正方论证过程:
</response>
"""


In [ ]:
neg_prompt = """
你是一个辩论场上的选手，你需要根据提供的反方观点进行论证。
保持语言简洁明了，避免过于复杂或模糊的表达。
确保论证严谨，避免情感化的语言或空洞的说法。
使用可靠的来源来支持论点，避免无根据的主张。

如果辩论对方有任何的观点意见，你需要回应对方的观点意见并修改自己的论证。你的目标不是赢，而是探寻真理。

你需要使用下面的XML标签来输出。

<response>
反方观点：
反方论证过程：
</response>
"""


In [ ]:
pos_task = "正方观点如下: AI的迅速发展会带来大量失业"
neg_task = "反方观点如下: AI的迅速发展不会带来大量失业"


In [ ]:
def pos_generate(prompt: str, task: str, context: str = "") -> str:
    full_prompt = f"{prompt}\n{context}\n{task}" if context else f"{prompt}\n{task}"
    response = call_llm(full_prompt)
    result = extract_xml(response, "response")

    print("\n=== POS START ===")
    print(f"Generated:\n{result}")
    print("=== POS END ===\n")

    return result


In [ ]:
def neg_generate(prompt: str, task: str, context: str = "") -> str:
    full_prompt = f"{prompt}\n{context}\n{task}" if context else f"{prompt}\n{task}"
    response = call_llm(full_prompt)
    result = extract_xml(response, "response")

    print("\n=== NEG START ===")
    print(f"Generated:\n{result}")
    print("=== NEG END ===\n")

    return result


In [ ]:
def loop(pos_task: str, neg_task: str, pos_prompt: str, neg_prompt: str) -> list:
    memory = []
    # 正方先输出
    result = pos_generate(pos_prompt, pos_task)
    memory.append(result)

    # 进入迭代
    for _ in range(3):
        # 拼接背景内容
        memory_string = "\n".join(memory)
        context = f"双方论证历史信息如下：\n{memory_string}"
        
        # 反方输出
        result = neg_generate(neg_prompt, context, neg_task)
        memory.append(result)

        # 拼接背景内容
        memory_string = "\n".join(memory)
        context = f"双方论证历史信息如下：\n{memory_string}"
        
        # 正方输出
        result = pos_generate(pos_prompt, context, pos_task)
        memory.append(result)

    return memory


In [ ]:
output = loop(pos_task, neg_task, pos_prompt, neg_prompt)


Orchestrator-Workers，它是由一个大模型作为管理协调者对复杂任务进行拆分，拆分成子任务以及任务说明，再分发给其它大模型进行下一步执行，它适合于无法在事前确定子任务个数的情况。


In [ ]:
from openai import OpenAI
import os
import re
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
llm_name = "qwen-plus"

def call_llm(user_prompt, system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = client.chat.completions.create(
        model=llm_name,
        messages=messages
    )
    return response.choices[0].message.content


In [ ]:
def extract_xml(text: str, tag: str) -> str:
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""


In [ ]:
def extract_tasks(xml_string):
    pattern = r'<task>(.*?)</task>'
    tasks = re.findall(pattern, xml_string, re.DOTALL)
    return tasks


In [ ]:
class FlexibleOrchestrator:
    """Break down tasks and run them in parallel using worker LLMs."""
    
    def __init__(self, orchestrator_prompt: str, worker_prompt: str):
        """Initialize with prompt templates."""
        self.orchestrator_prompt = orchestrator_prompt
        self.worker_prompt = worker_prompt

    def _format_prompt(self, template: str, **kwargs):
        """Format a prompt template with variables."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process(self, task: str, context=None):
        """Process task by breaking it down and running subtasks in parallel."""
        context = context or {}

        # Step 1: Get orchestrator response
        orchestrator_input = self._format_prompt(
            self.orchestrator_prompt,
            task=task
        )
        orchestrator_response = call_llm(orchestrator_input)

        # Parse orchestrator response
        analysis = extract_xml(orchestrator_response, "analysis")
        tasks_xml = extract_xml(orchestrator_response, "tasks")
        tasks = extract_tasks(tasks_xml)

        print("\n=== ORCHESTRATOR OUTPUT ===")
        print(f"\nANALYSIS:\n{analysis}")
        print(f"\nTASKS:\n{tasks}")

        # Step 2: Process each task
        worker_results = []
        for task_info in tasks:
            worker_input = self._format_prompt(
                self.worker_prompt,
                original_task=task,
                task_description=task_info
            )
            worker_response = call_llm(worker_input)
            result = extract_xml(worker_response, "response")

            worker_results.append({
                "description": task_info,
                "result": result
            })

            print(f"\n=== WORKER RESULT ({task_info}) ===\n{result}\n")

        return {
            "analysis": analysis,
            "worker_results": worker_results,
        }


In [ ]:
ORCHESTRATOR_PROMPT = """
你需要分析下面的问题，并将其分拆成几个不同的立场进行多视角分析。

需要分析的问题如下：{task}

返回格式输出示例如下：

<analysis>在这里解释你对需要分析的问题的理解。</analysis>

<tasks>
    <task>在这里输出视角1</task>
    <task>在这里输出视角2</task>
</tasks>
"""


In [ ]:
WORKER_PROMPT = """
基于如下要求进行回答：
你需要分析的问题如下：{original_task}
你的分析视角如下：{task_description}

返回格式示例如下：

<response>
在这里输出你的回答内容
</response>
"""


In [ ]:
orchestrator = FlexibleOrchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
)



In [ ]:
results = orchestrator.process(
    task="AI时代需要学习编程"
)


智能体的六种设计模式之六，ReAct Agent，它适合于开放性问题无法事先定义工作流，大模型自行规划执行条件和步骤，再根据外部反馈进行下一步行动，最终完成任务。

In [ ]:
from openai import OpenAI
import os
import re
from dotenv import load_dotenv
load_dotenv()



In [ ]:

system_prompt = """
You are a ReAct (Reasoning and Acting) agent that follows a loop of Thought, Action, PAUSE, and Observation to solve problems.

Workflow:
1. Thought: Describe your reasoning or plan for solving the problem.
2. Action: Execute an appropriate action based on your reasoning. The available actions are listed below.
3. PAUSE: Indicate that you are pausing to observe the result of the action. Stop output anything while pause.
4. Observation: Analyze the result of the action and incorporate it into your reasoning.

At the end of the loop, provide a final Answer based on the information gathered.

Your available actions are:
  
  calculate:
    e.g. calculate: 5*7/4
    run a calculation and returns the number using python so be sure to use floating point syntax if needed.

  planet_mass:
    e.g. planet_mass: Mars
    returns the mass of the planet in the solar system
"""

In [ ]:
def calculate(what):
    return eval(what)


In [ ]:
def planet_mass(planet):
    masses = {
        "Mercury": 0.3301,
        "Venus": 4.8557,
        "Earth": 5.972,
        "Mars": 0.6418,
        "Jupiter": 1898.2,
        "Saturn": 568.34,
        "Uranus": 86.82,
        "Neptune": 102.4,
    }
    return f"{planet} has a mass of {masses[planet]} × 10^24 kg"


In [ ]:
know_actions = {
    "calculate": calculate,
    "planet_mass": planet_mass
}


In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(system_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print("----------------------")
        print(f"step: {i}")
        print(result)
        action_re = re.compile(r"^Action:\s*(\w+):\s*(.*)$")
        actions = [action_re.match(line) for line in result.split("\n") if action_re.match(line)]
        if actions:
            print("\nlet's do something")
            action, action_input = actions[0].groups()
            if action not in know_actions:
                raise Exception(f"Unknown action: {action}")
            print(f"\nRunning {action}({action_input})")
            observation = know_actions[action](action_input)
            print(f"\nObservation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return

question = "what is the combined mass of earth and jupiter"
query(question)


智能体的六种设计模式之框架篇：用LangGraph实现chain

第一部分：https://www.bilibili.com/video/BV1FrwReHEo2?spm_id_from=333.788.videopod.sections&vd_source=680c14d8fc47569fbccdd26dfabf6b28

In [ ]:
import os
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()



In [ ]:
# llm = ChatOpenAI(
#     model='deepseek-chat',
#     api_key=os.getenv("DEEPSEEK_API_KEY"),
#     base_url='https://api.deepseek.com'
#     max_tokens=1024
# )


In [ ]:
llm = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
class State(TypedDict):
    topic: str
    outline: str
    paper: str
    final_paper: str


In [ ]:
writing_processing_steps = [
    """
    请根据以下主题撰写一篇议论文的大纲，
    包含引言、三到五个主要论点，以及结论。
    每个论点附带简要的解释或论据，
    主题如下：
    """,
    """
    请根据以下大纲撰写一篇完整的议论文，
    要求语言流畅，逻辑清晰，论点充分，
    每个段落围绕大纲内容展开，
    字数为1000字左右。
    大纲如下：
    """,
    """
    请对以下论文进行全文润色，包括语法检查、用词优化和句式调整，
    使文章语言自然流畅，逻辑清晰，表达生动而简洁，
    避免生硬和刻板的AI风格。保持原文观点不变。
    文章如下：
    """
]


In [ ]:
# Nodes
def generate_outline(state: State):
    prompt = writing_processing_steps[0] + state["topic"]
    msg = llm.invoke(prompt)
    return {"outline": msg.content}

def generate_paper(state: State):
    prompt = writing_processing_steps[1] + state["outline"]
    msg = llm.invoke(prompt)
    return {"paper": msg.content}

def polish_paper(state: State):
    prompt = writing_processing_steps[2] + state["paper"]
    msg = llm.invoke(prompt)
    return {"final_paper": msg.content}


In [ ]:
workflow = StateGraph(State)
workflow.add_node("generate_outline", generate_outline)
workflow.add_node("generate_paper", generate_paper)
workflow.add_node("polish_paper", polish_paper)
workflow.add_edge(START, "generate_outline")
workflow.add_edge("generate_outline", "generate_paper")
workflow.add_edge("generate_paper", "polish_paper")
workflow.add_edge("polish_paper", END)
graph = workflow.compile()


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = graph.invoke({"topic": "人工智能时代更要学习编程"})
print(state)


第二部分：https://www.bilibili.com/video/BV1h6AtecE8P?spm_id_from=333.788.videopod.sections&vd_source=680c14d8fc47569fbccdd26dfabf6b28

In [ ]:
import os
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()



In [ ]:
llm = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
# Graph state
class State(TypedDict):
    topic: str
    part0: str
    part1: str
    part2: str
    part3: str
    combined_output: str



In [ ]:
baijia = [
    "儒家",
    "法家",
    "道家",
    "墨家",
]


In [ ]:
meta_prompt = """
针对如下问题进行思考分析，需要一步步分析展示思考过程，并得出结论。
问题如下：{topic}。
你分析的框架和角度如下：{aspect}
"""


In [ ]:
# Nodes
def call_llm_0(state: State):
    topic = state["topic"]
    aspect = baijia[0]
    prompt = meta_prompt.format(topic=topic, aspect=aspect)
    msg = llm.invoke(prompt)
    return {"part0": msg.content}

def call_llm_1(state: State):
    topic = state["topic"]
    aspect = baijia[1]
    prompt = meta_prompt.format(topic=topic, aspect=aspect)
    msg = llm.invoke(prompt)
    return {"part1": msg.content}
def call_llm_2(state: State):
    topic = state["topic"]
    aspect = baijia[2]
    prompt = meta_prompt.format(topic=topic, aspect=aspect)
    msg = llm.invoke(prompt)
    return {"part2": msg.content}

def call_llm_3(state: State):
    topic = state["topic"]
    aspect = baijia[3]
    prompt = meta_prompt.format(topic=topic, aspect=aspect)
    msg = llm.invoke(prompt)
    return {"part3": msg.content}



In [ ]:
# Build workflow
parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("call_llm_0", call_llm_0)
parallel_builder.add_node("call_llm_1", call_llm_1)
parallel_builder.add_node("call_llm_2", call_llm_2)
parallel_builder.add_node("call_llm_3", call_llm_3)
parallel_builder.add_node("aggregator", aggregator)

# Add edges to connect nodes
parallel_builder.add_edge(START, "call_llm_0")
parallel_builder.add_edge(START, "call_llm_1")
parallel_builder.add_edge(START, "call_llm_2")
parallel_builder.add_edge(START, "call_llm_3")
parallel_builder.add_edge("call_llm_0", "aggregator")
parallel_builder.add_edge("call_llm_1", "aggregator")
parallel_builder.add_edge("call_llm_2", "aggregator")
parallel_builder.add_edge("call_llm_3", "aggregator")
parallel_builder.add_edge("aggregator", END)

parallel_workflow = parallel_builder.compile()


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = graph.invoke({"topic": "to be or not be"})
print(state)

第三部分：https://www.bilibili.com/video/BV1X3PYeEEoK?spm_id_from=333.788.videopod.sections&vd_source=680c14d8fc47569fbccdd26dfabf6b28

In [ ]:
import os
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()



In [ ]:
llm = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
writing_prompt = {
    "议论文": """
    撰写一篇议论文，围绕以下主题展开明确的观点，并提供有力的论据支持。
    要求：
    1. **观点鲜明**：开篇提出明确的论点。
    2. **逻辑严谨**：通过事实、数据、案例进行论证。
    3. **结构清晰**：包括引言、论证、结论三部分。
    4. **语言有说服力**：条理分明，语言简洁有力。
    """,
    
    "散文": """
    请写一篇散文，围绕以下主题抒发真挚细腻的情感，描绘生动的画面。
    要求：
    1. **情感真挚**：用细腻的文字表达内心感受。
    2. **意境优美**：通过比喻、拟人等修辞，营造氛围。
    3. **语言流畅**：语言自然，富有节奏感。
    4. **不拘结构**：可以自由组织段落和内容。
    """,
    
    "诗歌": """
    请创作一首诗歌，围绕以下主题，用简洁生动的语言表达情感或描绘画面。
    要求：
    1. **意象丰富**：用生动的意象表现主题。
    2. **语言凝练**：短句精练，富有节奏感。
    3. **情感浓烈**：真挚自然地传达内心情感。
    """,
    
    "小说": """
    请创作一篇小说，围绕以下主题展开故事情节，塑造生动的角色，并营造引人入胜的氛围。
    要求：
    1. **情节完整**：包括开端、发展、高潮、结局。
    2. **人物鲜明**：塑造富有个性和深度的角色。
    3. **氛围生动**：通过细节描写和环境铺垫增强代入感。
    4. **语言流畅**：叙述清晰，富有表现力。
    """
}


In [ ]:
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal['论文', '散文', '诗歌', '小说'] = Field(
        default=None, description="The next step in the routing process"
    )

# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


In [ ]:
class State(TypedDict):
    input: str
    decision: str
    output: str

def llm_call_1(state: State):
    prompt = f"{writing_prompt['议论文']} \n主题如下:{state['input']}"
    result = llm.invoke(prompt)
    return {"output": result.content}

def llm_call_2(state: State):
    prompt = f"{writing_prompt['散文']} \n主题如下:{state['input']}"
    result = llm.invoke(prompt)
    return {"output": result.content}

def llm_call_3(state: State):
    prompt = f"{writing_prompt['诗歌']} \n主题如下:{state['input']}"
    result = llm.invoke(prompt)
    return {"output": result.content}

def llm_call_4(state: State):
    prompt = f"{writing_prompt['小说']} \n主题如下:{state['input']}"
    result = llm.invoke(prompt)
    return {"output": result.content}

In [ ]:
def llm_call_router(state: State):
    decision = router.invoke(
        [
            SystemMessage(
                content="根据用户输入的主题，判断其需要的写作风格，并将内容路由到对应模块。"
            ),
            HumanMessage(content=state["input"]),
        ]
    )
    return {"decision": decision.step}

In [ ]:
def route_decision(state: State):
    if state["decision"] == "议论文":
        return "llm_call_1"
    elif state["decision"] == "散文":
        return "llm_call_2"
    elif state["decision"] == "诗歌":
        return "llm_call_3"
    elif state["decision"] == "小说":
        return "llm_call_4"

In [ ]:
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_4", llm_call_4)
router_builder.add_node("llm_call_router", llm_call_router)

# Add edges to connect nodes
router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
        "llm_call_4": "llm_call_4",
    },
)

router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)
router_builder.add_edge("llm_call_4", END)

# Compile workflow
router_workflow = router_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = router_workflow.invoke({"input": "天边的云"})
print(state['decision'])
print(state["output"])

第四部分：https://www.bilibili.com/video/BV1n79iY1Eg8?spm_id_from=333.788.videopod.sections&vd_source=680c14d8fc47569fbccdd26dfabf6b28

In [ ]:
import os
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()



In [ ]:
llm = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:

# Graph state
class State(TypedDict):
    topic: str
    paper: str
    feedback: str
    good_or_not: str


In [ ]:

from pydantic import BaseModel, Field
from typing_extensions import Literal

class Feedback(BaseModel):
    grade: Literal["合格", "不合格"] = Field(
        description="判断文章的逻辑性是否合格",
    )
    feedback: str = Field(
        description="根据文章的论证质量，给出修改建议",
    )


In [ ]:
evaluator = llm.with_structured_output(Feedback)

In [ ]:
def llm_call_generator(state: State):
    if state.get("feedback"):
        prompt = f"""根据提供的主题写一篇论证文章。确保文章逻辑严密、有说服力。
主题为：{state['topic']}。
你同时需要考虑如下的修改建议：{state['feedback']}"""
        msg = llm.invoke(prompt)
    else:
        prompt = f"""根据提供的主题写一篇论证文章。确保文章逻辑严密、有说服力。
主题为：{state['topic']}。"""
        msg = llm.invoke(prompt)

    count = state['count'] + 1
    return {"paper": msg.content, "count": count}

In [ ]:
def llm_call_evaluator(state: State):
    prompt = f"""根据提供的论证文章，判断是否逻辑严密、有说服力。
文章如下：{state['paper']}"""
    result = evaluator.invoke(prompt)
    return {"good_or_not": result.grade, "feedback": result.feedback}

In [ ]:
def route_paper(state: State):
    if state["good_or_not"] == "合格":
        return "Accepted"
    elif state['count'] >= 2:
        return "Accepted"
    elif state["good_or_not"] == "不合格":
        return "Rejected and Feedback"

In [ ]:
# Build workflow
optimizer_builder = StateGraph(State)

# Add the nodes
optimizer_builder.add_node("llm_call_generator", llm_call_generator)
optimizer_builder.add_node("llm_call_evaluator", llm_call_evaluator)

# Add edges to connect nodes
optimizer_builder.add_edge(START, "llm_call_generator")
optimizer_builder.add_edge("llm_call_generator", "llm_call_evaluator")
optimizer_builder.add_conditional_edges(
    "llm_call_evaluator",
    route_paper,
    {
        "Accepted": END,
        "Rejected and Feedback": "llm_call_generator",
    },
)

# Compile the workflow
optimizer_workflow = optimizer_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = optimizer_workflow.invoke({"topic": "AI的迅速发展会带来大量失业吗", 'count': 0})

In [ ]:
print(state["paper"])
print(state["good_or_not"])
print(state["feedback"])

第五部分：https://www.bilibili.com/video/BV1rVQgYvE6i?spm_id_from=333.788.videopod.sections&vd_source=680c14d8fc47569fbccdd26dfabf6b28

In [ ]:
import os
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()


In [ ]:
llm = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


In [ ]:
import operator

# Schema for structured output to use in planning
class Section(BaseModel):
    name: str = Field(
        description="章节标题",
    )
    description: str = Field(
        description="章节概述",
    )

class Sections(BaseModel):
    sections: List[Section] = Field(
        description="论证文章的各个子章节",
    )

# Augment the LLM with schema for structured output
planner = llm.with_structured_output(Sections)

In [ ]:
output = planner.invoke("论证主题：AI时代需要学习编程，将这个论证分拆成几个不同的立场进行多视角分析")



In [ ]:
output.sections

In [ ]:
from langgraph.constants import Send
from langchain.schema import HumanMessage, SystemMessage

# Graph state
class State(TypedDict):
    topic: str  # Report topic
    sections: list[Section]  # List of report sections
    completed_sections: Annotated[
        list, operator.add
    ]  # All workers write to this key in parallel
    final_report: str  # Final report

# Worker state
class WorkerState(TypedDict):
    section: Section
    completed_sections: Annotated[list, operator.add]

In [ ]:
def orchestrator(state: State):
    # Generate queries
    output = planner.invoke(
        [
            SystemMessage(content="你需要分析论证某个主题，并将其分拆成几个不同的立场进行多视角分析。"),
            HumanMessage(content=f"论证主题：{state['topic']}")
        ]
    )
    return {"sections": output.sections}

In [ ]:
def llm_call(state: WorkerState):
    # Generate section(content) str | list[str | dict]
    output = llm.invoke(
        content=[
            SystemMessage(
                content="根据提供的章节标题和章节概述，完成论证文章的其中一个章节"
            ),
            HumanMessage(
                content=f"章节标题为：{state['section'].name} 章节概述为：{state['section'].description}",
            ),
        ]
    )
    # Write the updated section to completed sections
    return {"completed_sections": [output.content]}

In [ ]:
def synthesizer(state: State):
    # List of completed sections
    completed_sections = state["completed_sections"]

    # Format completed section to str to use as context for final sections
    completed_report_sections = "\n\n---\n\n".join(completed_sections)

    return {"final_report": completed_report_sections}

In [ ]:
# Conditional edge function to create llm_call workers that each write a section of the report
def assign_workers(state: State):
    # Kick off section writing in parallel via Send() API
    return [Send("llm_call", {"section": s}) for s in state["sections"]]

In [ ]:
# Build workflow
orchestrator_worker_builder = StateGraph(State)

# Add the nodes
orchestrator_worker_builder.add_node("orchestrator", orchestrator)
orchestrator_worker_builder.add_node("llm_call", llm_call)
orchestrator_worker_builder.add_node("synthesizer", synthesizer)

# Add edges to connect nodes
orchestrator_worker_builder.add_edge(START, "orchestrator")
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator", assign_workers, ["llm_call"]
)
orchestrator_worker_builder.add_edge("llm_call", "synthesizer")
orchestrator_worker_builder.add_edge("synthesizer", END)

# Compile the workflow
orchestrator_worker = orchestrator_worker_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke
state = orchestrator_worker.invoke({"topic": "AI时代需要学习编程"})


In [ ]:
state["sections"]

In [ ]:
from IPython.display import Markdown
Markdown(state["final_report"])

第六部分:https://www.bilibili.com/video/BV1xqXGYDELg?spm_id_from=333.788.videopod.sections&vd_source=680c14d8fc47569fbccdd26dfabf6b28

智能体的六种设计模式之框架篇：用LangGraph实现ReAct Agent
1. 概述：

ReAct Agent是一种基于LangChain的AI代理设计模式，它通过结合ReAct和LangChain的Graph模块，实现了一个智能体，该智能体可以执行多个任务，并在执行过程中进行迭代优化。

2. ReAct Agent的组成：

ReAct Agent由三个部分组成：

- 智能体：一个AI代理，它包含一个模型和一组工具。
- 任务：一个描述智能体要执行的任务的文本。

In [ ]:
import os
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

load_dotenv()


In [ ]:
llm = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",)

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Adds a and b.

    Args:
        a: first float
        b: second float
    """
    return a + b

@tool
def planet_mass(planet):
    """Return the mass of a planet, args: planet name(capitalized), e.g. Mercury"""
    masses = {
        "Mercury": 0.3301,
        "Venus": 4.8557,
        "Earth": 5.972,
        "Mars": 0.6418,
        "Jupiter": 1898.2,
        "Saturn": 568.34,
        "Uranus": 86.82,
        "Neptune": 102.4
    }
    return f"{planet} has a mass of {masses[planet]} 10^24 kg"

tools = [add, planet_mass]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)



In [ ]:
tools_by_name

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langgraph.prebuilt import ToolNode

# Nodes
def llm_call(state: MessagesState):
    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    SystemMessage(content="You are a helpful assistant.")
                ] + state["messages"]
            )
        ]
    }
# def tool_node(state: MessagesState):
#     result = []
#     for tool_call in state["messages"][-1].tool_calls:
#         tool = tools_by_name[tool_call["name"]]
#         observation = tool.invoke(tool_call["args"])
#         result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
#     return {"messages": result}

tool_node = ToolNode(tools=tools)

# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "Action"
    
    # Otherwise, we stop (reply to the user)
    return END


In [ ]:
# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tools", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        # Name returned by should_continue : Name of next node to visit
        "Action": "tools",
        END: END,
    },
)

agent_builder.add_edge("tools", "llm_call")
agent_builder.agent_builder.complile()

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph().draw_mermaid_png()))


In [ ]:
# Invoke
messages = [HumanMessage(content="what is the combined mass of earth and jupiter")]
messages = agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()


In [ ]:
messages['message']